In [1]:
%pip install pandas numpy matplotlib

In [4]:
import pandas as pd
df = pd.read_csv('data_train.csv', encoding='latin1')
df = df.sample(frac=0.01, random_state=42)
df.dropna(inplace=True)
questions = df.question1.values

In [5]:
questions[:5]

array(['How do I play Pokémon GO in Korea?',
       'What are some of the best side dishes for crab cakes?',
       'Which is more advisable and better material for a crash test in automobiles, ductile or brittle?',
       'How do I improve logical programming skills?',
       'How close we are to see 3rd world war?'], dtype=object)

In [6]:
%pip install tensorflow tensorflow-hub

   ---------------------------------------- 0.0/331.8 MB ? eta -:--:--
   ---------------------------------------- 2.1/331.8 MB 10.7 MB/s eta 0:00:31
    --------------------------------------- 5.0/331.8 MB 11.2 MB/s eta 0:00:30
    --------------------------------------- 7.3/331.8 MB 11.3 MB/s eta 0:00:29
   - -------------------------------------- 10.0/331.8 MB 11.5 MB/s eta 0:00:28
   - -------------------------------------- 12.6/331.8 MB 11.4 MB/s eta 0:00:28
   - -------------------------------------- 14.9/331.8 MB 11.5 MB/s eta 0:00:28
   -- ------------------------------------- 17.0/331.8 MB 11.4 MB/s eta 0:00:28
   -- ------------------------------------- 19.4/331.8 MB 11.5 MB/s eta 0:00:28
   -- ------------------------------------- 21.8/331.8 MB 11.3 MB/s eta 0:00:28
   -- ------------------------------------- 23.6/331.8 MB 11.1 MB/s eta 0:00:28
   --- ------------------------------------ 25.7/331.8 MB 10.9 MB/s eta 0:00:29
   --- ------------------------------------ 28.0/331


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\kotag\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
%pip install flair

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 981.5/981.5 kB 5.1 MB/s  0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -----------------

  DEPRECATION: Building 'langdetect' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'langdetect'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'pptree' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pptree'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'sqlitedict' using the legacy setup.py bdist_wheel mecha

In [ ]:
# install flair if not already available
from abc import ABCMeta, abstractmethod
import tensorflow_hub as hub
import tensorflow as tf

# flair imports (alias TransformerDocumentEmbeddings to BERT)
from flair.embeddings import TransformerDocumentEmbeddings
from flair.data import Sentence

class TFEncoder(metaclass=ABCMeta):
    def __init__(self,model_path):
        self.model=hub.load(model_path)
    @abstractmethod
    def encode(self,text):
        pass
class UniversalSentenceEncoder(TFEncoder):
    def __init__(self,model_path):
        super().__init__(model_path)
    def encode(self,text):
        return self.model(text).numpy()
class USEQA(TFEncoder):
    def __init__(self,model_path):
        super().__init__(model_path)
    def encode(self,text):
        return self.model.signatures['question_encoder'](tf.constant(text))['outputs'].numpy()
class BERT():
    def __init__(self, model_name='xlm-roberta-base', layers='-2', pooling_operation="mean"):
        self.embeddings = TransformerDocumentEmbeddings(model_name)
    
    def encode(self, text):
        sentence = Sentence(text)
        self.embeddings.embed(sentence)
        return sentence.get_embedding().cpu().detach().numpy()

In [11]:
import faiss
class FAISS:
    def __init__(self,dimensions):
        self.dimensions=dimensions
        self.index=faiss.IndexFlatL2(dimensions)
        self.vectors={}
        self.counter=0
    def add(self,text,v):
        self.index.add(v)
        self.vectors[self.counter]=(text,v)
        self.counter+=1
    def search(self,v,k):
        distance,item_idx=self.index.search(v,k)
        for d,i in zip(distance[0],item_idx[0]):
            if i==-1:
                break
            text,vec=self.vectors[i]
            print(f"Distance: {d:.4f} \t Text: {text}")

In [ ]:
#encode and index the questions
import numpy as np
encoder=BERT('bert-base-uncased')
dimension=encoder.encode("sample question").shape[0]
faiss_index=FAISS(dimension)
for question in questions:
    vec=encoder.encode(question)
    vec=vec.reshape(1,-1)
    faiss_index.add(question,vec)
# Enhanced search with multilingual support
query = "How to learn machine learning?"  # Works in any language
detected_lang = detect_language(query)
print(f"Query detected language: {detected_lang}")

query_vec = encoder.encode(query).reshape(1,-1)
faiss_index.search(query_vec, 5)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\kotag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kotag\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
%pip install langdetect
from langdetect import detect, detect_langs

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

In [ ]:
# Filter and handle multilingual data
import pandas as pd
df = pd.read_csv('data_train.csv', encoding='latin1')
df['detected_language'] = df['question1'].apply(detect_language)
print(df['detected_language'].value_counts())